# Introduction
The CIA Wolrd Factbook is a compendium of statisitcs about all of the countries on Earth. It contains demographic information such as:

- `population` - the global population.
- `population_growth` - the annual population growth rate, as a percentage.
- `area` - the total land and water area.

This project will utilize SQL to analyze data from this database. 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

# here we can see the table name is 'facts'

Done.
(sqlite3.OperationalError) unrecognized token: "#"
[SQL: # here we can see the table name is 'facts']
(Background on this error at: http://sqlalche.me/e/13/e3q8)


Let's inspect the first five rows of the `facts` table

In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are some descriptions of the variable names:

- `name` — the name of the country.
- `area` — the country's total area (both land and water).
- `area_land` — the country's land area in square kilometers.
- `area_water` — the country's waterarea in square kilometers.
- `population` — the country's population.
- `population_growth` — the country's population growth as a percentage.
- `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

Let's run some descriptive statsistics to look for any outlier countries on the population variables.

In [4]:
%%sql
SELECT MIN(population) AS min_population,
       MAX(population) AS max_population,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
  FROM facts; 

Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


Weird, looks like there is a country with a population of zero. That can't be right. There's also a country with an excess of 7 billion people.

Let's use some queries to specifically inspect these countries.

In [5]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                       FROM facts
                    );



Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


Looks like this country with no population is Antarctica!

In [6]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                       FROM facts
                    );

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


...and it is no country that has a population this large, but the whole world.

We can check that against computing the actual sum of the population variable in the database (excluding `World`)

In [7]:
%%sql
SELECT SUM(population)
  FROM facts
 WHERE name != 'World';

Done.


SUM(population)
7770482643


Looks like my hypothesis was wrong - this computed value is bigger by about 500 million. It could be that the database includes other aggregates by continent, group, etc.

Let's inspect each individual name in the dataset and their respective population sums

In [8]:
%%sql
SELECT name, SUM(population) AS population
  FROM facts
 GROUP BY name

Done.


name,population
Afghanistan,32564342
Akrotiri,15700
Albania,3029278
Algeria,39542166
American Samoa,54343
Andorra,85580
Angola,19625353
Anguilla,16418
Antarctica,0
Antigua and Barbuda,92436


It's a lot to wade through, but there does appear to be some overlap in these territories, which would explain the mismatch between the world and calculated population sums.

Now, let's recalculate some summary statistics while excluding the 'World' case, which isn't a country.

In [9]:
%%sql
SELECT MIN(population) AS min_population,
       MAX(population) AS max_population,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
  FROM facts
 WHERE name != 'World';

Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


Now, the max is about 1.4 billion. Let's calculate the averages.

In [10]:
%%sql
SELECT AVG(population), AVG(area)
  FROM facts;

Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


This next code block will find countries with dense populations, as defined by having above-average populations and below-average areas.

In [12]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name != 'World')
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name != 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31
